Stress the importance of noise
- gaussian noise
- uniform noise
- cauchy noise
- linear noise <br>
https://en.wikipedia.org/wiki/Noise_(signal_processing)

#Stochastic Hopfield networks

The stochastic Hopfield network is closely related to systems studied in statistical mechanics, so-called spin glasses (exhibits order-disorder transition -  becomes sharp in the limit of large N).

**Noisy dynamics**

The update rule:

$$S_i \leftarrow sgn (\sum_j w_{ij} S_j - \theta_i) $$
can be written as

$$S_i \leftarrow sgn (b_i)$$ (1)

where $b_i$ is the local field. This rule is called deterministic, because a given set of states $S_i$ determines the outcome of the update. 

 To introduce noise, one replaces the rule (1) by the stochastic rule:
 
 $$
S_i = 
  \begin{cases}
    +1 & \text{with probability} P(b_i)\\
    -1 & \text{with probability} 1 - P(b_i) 
  \end{cases}
.
 $$

       
 where
 
$$ P(b) = \frac{1}{1+e^{-2 \beta b}}$$

The parameter $\beta$ is the noise parameter.

In particular one obtains the deterministic dynamics as $\beta$ tends to infinity. Conversely, when β = 0, the function $P(b)$ simply equals $1/2$.
In this case $S_i$ is updated randomly with equal probability to −1 or +1. 

The stochastic algorithm is very similar to the deterministic
algorithm for the Hopfield model (Algorithm 1), only step 4 is different

**Algorithm 1: pattern recognition with deterministic Hopfield model**

1: store patterns $x^{\mu}$ using Hebb’s rule;

2: feed distorted pattern x into network by assigning $S_i(t = 0) ← xi$;

3: for $ t = 1,...,T$ do
            4: choose a value of $i$ and update $S_i(t ) ←\sum_j w_{ij}S_j(t − 1)$;
            
5: end for

6: read out pattern $S(T)$;

7: end;

The idea is to run the network for a small but finite noise level, that is at large
value of β. Then the dynamics is very similar to the deterministic Hopfield dynamics
analysed in the previous Chapter. But the noise allows the system to sometimes
also go uphill, making it possible to escape spurious minima.

In [ ]:
to be continued...